In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [40]:
def accent_remover(x):
    try:
        float(x)
        return x
    except:
        return normalize('NFKD',x).encode('ASCII', 'ignore').decode('ASCII')

In [41]:
# Importar dataset
dataset = pd.read_csv('datasets/datatran2017.csv', sep= ',', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset = dataset.transform([accent_remover])

/home/maiconmares/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
# Pega as dados das colunas:
# data_inversa	dia_semana	horario	uf	br	km	condicao_metereologica	latitude	longitude
X = dataset.iloc[:, 11:17].values
y = dataset.iloc[:, 22].values
y=np.array(y)
print(X)
print(y)

[['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Garoa/Chuvisco'
  'Simples' 'Reta']
 ['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Chuva' 'Simples'
  'Nao Informado']
 ['Com Vitimas Fatais' 'Plena Noite' 'Decrescente' 'Ceu Claro' 'Simples'
  'Curva']
 ...
 ['Com Vitimas Fatais' 'Anoitecer' 'Decrescente' 'Nublado' 'Simples'
  'Curva']
 ['Com Vitimas Feridas' 'Plena Noite' 'Crescente' 'Nublado' 'Simples'
  'Reta']
 ['Com Vitimas Feridas' 'Anoitecer' 'Crescente' 'Nublado' 'Dupla' 'Reta']]
[16  8  7 ... 30  8 13]


In [72]:
# Converte os dados de string para number gerando as categorias
encoder = LabelEncoder()
X[:,0] = encoder.fit_transform(X[:,0])
X[:,1] = encoder.fit_transform(X[:,1])
X[:,2] = encoder.fit_transform(X[:,2])
X[:,3] = encoder.fit_transform(X[:,3])
X[:,4] = encoder.fit_transform(X[:,4])
X[:,5] = encoder.fit_transform(X[:,5])
print(X)

#ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3])], remainder='passthrough')
#X=np.array(ct.fit_transform(X))
print(X)

[[1 2 1 2 2 5]
 [1 2 1 1 2 3]
 [0 2 1 0 2 0]
 ...
 [0 1 1 6 2 0]
 [1 2 0 6 2 5]
 [1 1 0 6 0 5]]
[[1 2 1 2 2 5]
 [1 2 1 1 2 3]
 [0 2 1 0 2 0]
 ...
 [0 1 1 6 2 0]
 [1 2 0 6 2 5]
 [1 1 0 6 0 5]]


In [80]:
# Divide o dataset em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [81]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
np.shape(X_train)

(35825, 6)

In [82]:
import tensorflow as tf
def my_leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.01)

In [94]:
# Inicializa a ANN
regressor = Sequential()

# Adiciona a primeira camada
regressor.add(Dense(units=6, activation='relu', input_dim=6))
regressor.add(Dropout(0.2))

# Segunda camada
regressor.add(Dense(units=6, activation='relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=6, activation='relu'))

# Camada de saida
regressor.add(Dense(units=1, activation='sigmoid'))
#regressor.add(Dropout(0.2))

# Compila
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['mean_squared_error', 'accuracy'])

# Executa o treinamento
regressor.fit(X_train, y_train, epochs = 100, batch_size = 25)


Epoch 1/100
1433/1433 [==============================] - 2s 1ms/step - loss: -22429.0938 - mean_squared_error: 309.3260 - accuracy: 0.0000e+00
Epoch 2/100
1433/1433 [==============================] - 2s 1ms/step - loss: -623999.8125 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 3/100
1433/1433 [==============================] - 2s 1ms/step - loss: -3308031.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 4/100
1433/1433 [==============================] - 2s 1ms/step - loss: -9783999.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 5/100
1433/1433 [==============================] - 2s 1ms/step - loss: -22104624.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 6/100
1433/1433 [==============================] - 2s 1ms/step - loss: -43046196.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 7/100
1433/1433 [==============================] - 2s 1ms/step - loss: -74581800.0000 - mean_squared_error: 307.3067 - accura

1433/1433 [==============================] - 2s 1ms/step - loss: -122962345984.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 57/100
1433/1433 [==============================] - 2s 1ms/step - loss: -131145490432.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 58/100
1433/1433 [==============================] - 2s 1ms/step - loss: -139852316672.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 59/100
1433/1433 [==============================] - 2s 1ms/step - loss: -149403516928.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 60/100
1433/1433 [==============================] - 2s 1ms/step - loss: -158589976576.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 61/100
1433/1433 [==============================] - 2s 1ms/step - loss: -169250504704.0000 - mean_squared_error: 307.3067 - accuracy: 0.0000e+00
Epoch 62/100
1433/1433 [==============================] - 2s 1ms/step - loss: -180010549248.0000 - mean_squ

In [ ]:
# Executa predição
y_pred=regressor.predict(X_test)
y_pred

In [ ]:
y_pred=(y_pred>0.5)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
print(cm)